# This notebook demos a diffusion stencil

We start with some imports & a small helper function to convert our stencil to SIR:

In [ ]:
from typing import Callable

from inspect import getsource
import ast

from dusk.grammar import Grammar

from dawn4py import compile, CodeGenBackend
from dawn4py.serialization import make_sir, SIR
from dawn4py.serialization.SIR import GridType
from dawn4py._dawn4py import run_optimizer_sir


def dusk_to_sir(stencil: Callable) -> SIR:
    # this will give wrong line numbers, there should be a way to fix them
    name = stencil.__name__
    stencil = ast.parse(getsource(stencil))

    assert isinstance(stencil, ast.Module)
    assert len(stencil.body) == 1
    stencil = stencil.body[0]
    assert Grammar.is_stencil(stencil)

    return make_sir(
        name, GridType.Value("Unstructured"), [Grammar().stencil(stencil)]
    )

With the above definitions, we can write our diffusion stencil:

In [ ]:
from dusk.script import *


@stencil
def diffusion(
    TV: Field[Vertex],
    TE: Field[Edge], TEinit: Field[Edge], TE_t: Field[Edge], TEnabla2: Field[Edge],
    inv_primal_edge_length: Field[Edge], inv_vert_vert_length: Field[Edge], nnbhV: Field[Vertex],
    boundary_edge: Field[Edge],
    kappa: Field[Edge], dt: Field[Edge]
) -> None:

    with levels_upward:
        #TODO replace with diffusion solver
        pass    


Then we can use dusk to convert the stencil to SIR. With dawn we can compile SIR to C++ which we will write to `diffusion_cxx-naive.cpp`:

In [ ]:
sir = dusk_to_sir(diffusion)
cpp_naive = compile(sir, backend=CodeGenBackend.CXXNaiveIco)
with open("diffusion_cxx-naive.cpp", "w+") as f:
    f.write(cpp_naive)

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [ ]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [ ]:
!./runner test

If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize

In [ ]:
!./runner run

In [ ]:
%%capture
import prep_animation as pa
import matplotlib.animation as animation
from IPython.display import HTML
animate = animation.FuncAnimation(pa.fig, pa.update_plot, len(pa.out_files), fargs=(pa.zarray, pa.plot), interval = 50)

In [ ]:
HTML(animate.to_html5_video())